# Cleaning Baltimore Crime Data with Pandas and Analyzing It with Tableau

### This notebook records my progress in cleaning the data of an Excel file and re-uploading it. The data will then go in to Tableau for some interesting visualizations and analysis.
### If you would like to download the pre-cleaned and finished spreadsheets, simply navigate to the 'BaltCrime' folder under 'other' in the folder of my github.io page.
### You can find the link to my [Tableau visualizations here.](https://public.tableau.com/app/profile/jacob.levy3764/viz/BaltimoreCrimeVisualized/Map)

In [48]:
import pandas as pd
#For the beginning of this notebook we'll focus on the first sheet, titled "Part1_Crime_data"
first_sheet_df = pd.read_excel("Baltimore Crime.xlsx", sheet_name=0)  
inside_outside_df = first_sheet_df['Inside_Outside']
print(inside_outside_df.unique())

[nan 'Outside' 'Inside' 'I' 'O']


Alright, so it's got 5 values. NaN, Inside, Outside, I, O. I want 3: Inside, Outside, and NA.  

In [49]:
#Making the O's into Outside, I's into Inside, and fill the null stuff with NA, to better match the rest of the dataset.
first_sheet_df['Inside_Outside']= inside_outside_df.replace({'O':'Outside', 'I':'Inside'}).fillna('NA')
#Let's see if it's 3 unique entries now.
print(first_sheet_df['Inside_Outside'].unique())

['NA' 'Outside' 'Inside']


Excellent. Next up we have to split one row into two. CrimeDateTime has the date and the time of the crime, and I'd like date and time to be their own columns to match up better with the dataset. You'll see when I use the .head() method.

In [50]:
df = first_sheet_df['CrimeDateTime']
splitTimeDate = df.str.split(expand=True).rename(columns = {0:'CrimeDate', 1:'CrimeTime'})
#Showing the newly split and renamed columns.
print(splitTimeDate.head())

    CrimeDate    CrimeTime
0  2022/02/19  07:30:00+00
1  2022/02/19  03:22:23+00
2  2022/02/19  11:20:00+00
3  2022/02/19  01:55:00+00
4  2022/02/19  10:00:00+00


Great! But CrimeTime is in 24 hour time, when I want it in 12 hour (or AM/PM) time. 
Thankfully, pandas has a way of doing just that.

In [51]:
#Formatting the time format to 12 hour time.
splitTimeDate['CrimeTime'] = pd.to_datetime(splitTimeDate['CrimeTime']).dt.strftime('%I:%M:%S %p')
print(splitTimeDate['CrimeTime'].head(5))

0    07:30:00 AM
1    03:22:23 AM
2    11:20:00 AM
3    01:55:00 AM
4    10:00:00 AM
Name: CrimeTime, dtype: object


Awesome, now we're going to combine the dataframe we've been working on with the original, and in the process drop the CrimeDateTime series from there.
While we're at it, we're going to drop a few more columns that aren't relevant to our work, and don't have an equivalent in the other sheet.

In [52]:
#Out with the old...
first_sheet_df.drop(columns = ['CrimeDateTime', 'X', 'Y', 'RowID', 'VRIName', 'Shape', 'GeoLocation', 'Total_Incidents'], axis = 1, inplace = True)
#And in with the new!
first_sheet_df['CrimeTime'] = splitTimeDate['CrimeTime']
first_sheet_df['CrimeDate'] = splitTimeDate['CrimeDate']
#And while we're at it, let's rename Inside_Outside to better match the other sheet's equivalent column.
first_sheet_df.rename(columns = {'Inside_Outside':'Inside/Outside'}, inplace = True)
print(first_sheet_df.columns.values)

['CrimeCode' 'Location' 'Description' 'Inside/Outside' 'Weapon' 'Post'
 'District' 'Neighborhood' 'Latitude' 'Longitude' 'Premise' 'CrimeTime'
 'CrimeDate']


Alright, so we've got our first sheet cleaned and ready for analysis. However, I want to combine it with the second sheet, which records data from 2015 - 2016. 
The goal here is to merge the columns seamlessly into one dataframe. 

In [53]:
second_sheet_df = pd.read_excel("Baltimore Crime.xlsx", sheet_name=1) 
#Well, it has some empty columns and data we're not really interested in, so we're gonna drop it. 
second_sheet_df.drop(columns = ['Location 1', 'crimeCaseNumber', 'Total Incidents'], axis=1,  inplace = True)

In [54]:
#Casting the CrimeDate column to a more appropriate datatype. This will smooth things over when we merge the sheets.
second_sheet_df['CrimeDate'] = second_sheet_df['CrimeDate'].dt.date
#And just like earlier, we'll have to fix up the Inside/Outside column to have only three unique values.
second_sheet_df['Inside/Outside'] = second_sheet_df['Inside/Outside'].replace({'O':'Outside', 'I':'Inside'}).fillna('NA')
print(second_sheet_df['Inside/Outside'].describe())

count       96946
unique          3
top       Outside
freq        48987
Name: Inside/Outside, dtype: object


In [55]:
combined_sheet_df = pd.concat([first_sheet_df, second_sheet_df])
#Just a few more things...
#Tableau is picky with location stuff. We're going to add the Country, State, and City as columns to the dataframe. It'll let Tableau read things better.
combined_sheet_df['Country'] = 'United States'
combined_sheet_df['State'] = 'Maryland'
combined_sheet_df['City'] = 'Baltimore'
#Let's see our columns now.
print(combined_sheet_df.columns.values)

#I'm also going to fix up the area names to be more consistent.
columnsNeedingTitle = ['Neighborhood', 'District', 'Location', 'Description', 'Weapon', 'Premise']
for column in columnsNeedingTitle:
    combined_sheet_df[column] = combined_sheet_df[column].str.title()

#And there we have it.
#If you're trying this out for yourself, this is going to take a while. It's (tragically) a lot of data.
combined_sheet_df.to_excel("Updated Baltimore Crime.xlsx", sheet_name='Crimes')

['CrimeCode' 'Location' 'Description' 'Inside/Outside' 'Weapon' 'Post'
 'District' 'Neighborhood' 'Latitude' 'Longitude' 'Premise' 'CrimeTime'
 'CrimeDate' 'Country' 'State' 'City']


## Thanks to pandas, manipulating this data was wonderfully simple. We can now see the fruits of our labor visualized through Tableau. [Click here to see the three visualizations I made!](https://public.tableau.com/app/profile/jacob.levy3764/viz/BaltimoreCrimeVisualized/Map)